In [22]:
import numpy as np
import numpy.ma as ma
import torch

In [23]:
data_a = np.array([[0,0,0],[10.2,0.1,0.1],[0,0,0]])
data_b = np.array([[0.1,0.1,0.1],[0.1,0.1,0.1],[0.1,0.1,0.1]])

In [24]:
masked_a = ma.masked_equal(data_a,0)
masked_a

masked_array(
  data=[[--, --, --],
        [10.2, 0.1, 0.1],
        [--, --, --]],
  mask=[[ True,  True,  True],
        [False, False, False],
        [ True,  True,  True]],
  fill_value=0.0)

In [25]:
masked_b = ma.masked_array(data_b,masked_a.mask,fill_value = 0)
masked_b

masked_array(
  data=[[--, --, --],
        [0.1, 0.1, 0.1],
        [--, --, --]],
  mask=[[ True,  True,  True],
        [False, False, False],
        [ True,  True,  True]],
  fill_value=0.0)

In [26]:
masked_b.filled()

array([[0. , 0. , 0. ],
       [0.1, 0.1, 0.1],
       [0. , 0. , 0. ]])

In [27]:
z = masked_a.mask

In [28]:
 (n_i, n_j) = z.shape


In [29]:
for i in range(n_i):
    for j in range(n_j):
        if z[i,j] == False:
            print(z[i,j])

False
False
False


In [30]:
to = torch.tensor(masked_b)

In [31]:
to

tensor([[0.1000, 0.1000, 0.1000],
        [0.1000, 0.1000, 0.1000],
        [0.1000, 0.1000, 0.1000]], dtype=torch.float64)

In [32]:
def search_edge(input,output,point_i,point_j, height, width):
    #识别边缘的点，并将缓冲区域内的点值更改为False
    if point_i - height < 0:
        start_i = 0
    else:
        start_i = point_i - height
    if point_j - width < 0:
        start_j = 0
    else:
        start_j = point_j - width
    if point_i + height+1 >= input.shape[0]:
        end_i = -1
    else:
        end_i =  point_i + height+1
    if point_j + width+1 >= input.shape[1]:
        end_j = -1
    else:
        end_j =point_j + width+1
    output[point_i,point_j] = input[start_i:end_i,start_j:end_j].all()
    
def mask_buffer(input_mask,height,width):
    #根据原始mask的边缘，通过缓冲区创建更大大的mask
    output_mask = input_mask.copy()
    for i in range(input_mask.shape[0]):
        for j in range(input_mask.shape[1]):
            search_edge(input_mask,output_mask,i,j,height,width)
    return output_mask

def area_extract(target_area, extract_area, buffer_height,buffer_width):
    #通过目标区域的buffered mask 提取下一时刻的研究区域
    #目标区域的掩码识别值为0，当值为0时，掩码值为True.否则为False
    #buffer 方式为矩形buffer,根据每一个边缘点的矩形范围创建buffer
    target_masked = ma.masked_equal(target_area,0)
    input_mask = target_masked.mask
    buffered_mask = mask_buffer(input_mask,buffer_height,buffer_width)
    extracted_area = ma.masked_array(extract_area, buffered_mask, fill_value = 0)
    extracted_area = extracted_area.filled()
    return extracted_area
    

In [33]:
a = torch.zeros((10,10))
a[4:6,3:7] = 5
a

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 5., 5., 5., 5., 0., 0., 0.],
        [0., 0., 0., 5., 5., 5., 5., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [34]:
b = torch.ones((10,10))
b

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [35]:
b_ = area_extract(a,b,1,1)
b_

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 1., 1., 1., 1., 0., 0.],
       [0., 0., 1., 1., 1., 1., 1., 1., 0., 0.],
       [0., 0., 1., 1., 1., 1., 1., 1., 0., 0.],
       [0., 0., 1., 1., 1., 1., 1., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [15]:
import numpy as np

In [16]:
a = np.array([1,2,3,4])

In [17]:
a = a.reshape(2,2)

In [18]:
a

array([[1, 2],
       [3, 4]])

In [19]:
b = np.tile(a,(3,1,1,1))

In [20]:
b

array([[[[1, 2],
         [3, 4]]],


       [[[1, 2],
         [3, 4]]],


       [[[1, 2],
         [3, 4]]]])

In [21]:
b.shape

(3, 1, 2, 2)